In [ ]:
df = sns.load_dataset('titanic')
df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [ ]:
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
df.isna().sum()

,0
survived,0
pclass,0
sex,0
age,177
sibsp,0
parch,0
fare,0
embarked,2
class,0
who,0


Попробуем заполнить пропуски так, чтобы не произошла утечка данных. Для этого разделим данные на обучающую и тестовую выборки и вычислим необходимые для заполнения пропусков значения только на тестовой выборке, применив затем это на тестовую выборку. Так мы не допустим попадения хотя бы какой-то информации из теста в обучение.

In [ ]:
from sklearn.model_selection import train_test_split

X = df.drop('survived', axis = 1)
y = df['survived']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((712, 14), (712,), (179, 14), (179,))

In [ ]:
from sklearn.impute import SimpleImputer

input = SimpleImputer(strategy = 'most_frequent')
X_train_fill = input.fit_transform(X_train)
X_test_fill = input.transform(X_test)

X_train = pd.DataFrame(X_train_fill, columns = X_train.columns, index = X_train.index)
X_test = pd.DataFrame(X_test_fill, columns = X_test.columns, index = X_test.index)
X_train.isna().sum(), X_test.isna().sum()

(pclass         0
 sex            0
 age            0
 sibsp          0
 parch          0
 fare           0
 embarked       0
 class          0
 who            0
 adult_male     0
 deck           0
 embark_town    0
 alive          0
 alone          0
 dtype: int64,
 pclass         0
 sex            0
 age            0
 sibsp          0
 parch          0
 fare           0
 embarked       0
 class          0
 who            0
 adult_male     0
 deck           0
 embark_town    0
 alive          0
 alone          0
 dtype: int64)

In [ ]:
X_train.shape

(712, 14)

Далее рассмотрим признаки, из  которых состоит набор данных попробуем найти те, которые могут в себе содержать информацию, тесно связанную с таргетом.

In [ ]:
df.columns

Index(['survived', 'pclass', 'sex', 'age', 'sibsp', 'parch', 'fare',
       'embarked', 'class', 'who', 'adult_male', 'deck', 'embark_town',
       'alive', 'alone'],
      dtype='object')

In [ ]:
df['adult_male'].unique()
df['alone'].unique()

array([False,  True])

Полностью коррелирует с целевой переменной стобец alone - он ее повторяет, поэтому от него мы избавляемся. Так же меня смущают столбцы, которые повторяют информацию. Например "sex" и "who", "alone" и "sibsp", "parch", также "embarked" и "embark_town". "adalt_male" коррелирует со стобцом возраста. Но вроде они не содержат информацию из целевой переменной, поэтому пока оставим их.

In [ ]:
X_train = X_train.drop('alive', axis = 1)
X_test = X_test.drop('alive', axis = 1)

In [ ]:
X_train_dop = X_train.copy()
X_test_dop = X_test.copy()

In [ ]:
from sklearn.preprocessing import OneHotEncoder

enc = OneHotEncoder()
X_train = enc.fit_transform(X_train[['sex', 'embarked', 'class', 'who', 'embark_town', 'deck']])
X_test = enc.transform(X_test[['sex', 'embarked', 'class', 'who', 'embark_town', 'deck']]) # Исправила здесь команду для предоствращения утечки

In [ ]:
X_train.shape

(712, 21)

In [ ]:
from sklearn.tree import DecisionTreeClassifier

drc = DecisionTreeClassifier(max_depth=10,
    min_samples_split=5,
    min_samples_leaf=3,
    random_state=42
    )
drc.fit(X_train, y_train)
y_pred = drc.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix

In [ ]:
acc_score = accuracy_score(y_test, y_pred)
prec_score = precision_score(y_test, y_pred)
rec_score = recall_score(y_test, y_pred)

print(f'Accuracy: {acc_score}, Precision: {prec_score}, Recall: {rec_score}')

Accuracy: 0.7932960893854749, Precision: 0.7681159420289855, Recall: 0.7162162162162162


In [ ]:
y_pred_prob = drc.predict_proba(X_test)[:, 1]
roc_auc = roc_auc_score(y_test, y_pred_prob)
print("ROC AUC:", roc_auc)

ROC AUC: 0.8718146718146719


In [ ]:
conf_mat = confusion_matrix(y_test, y_pred)
conf_mat

array([[89, 16],
       [21, 53]])

В целом, оценки метрик не плохие, но они далеки от идеала, поэтому попробуем более простую модель - Логистическую регрессию. Возможно, при использовании деревьев решений были подобраны не очень удачные параметры.

In [ ]:
from sklearn.linear_model import LogisticRegression

lg = LogisticRegression()
lg.fit(X_train, y_train)
y_pred_lg = lg.predict(X_test)
y_pred_lg_prob = lg.predict_proba(X_test)[:, 1]

In [ ]:
acc_score_lg = accuracy_score(y_test, y_pred_lg)
prec_score_lg = precision_score(y_test, y_pred_lg)
rec_score_lg = recall_score(y_test, y_pred_lg)

print(f'Accuracy: {acc_score_lg}, Precision: {prec_score_lg}, Recall: {rec_score_lg}')

Accuracy: 0.776536312849162, Precision: 0.7236842105263158, Recall: 0.7432432432432432


In [ ]:
roc_auc_lg = roc_auc_score(y_test, y_pred_lg_prob)
print("ROC AUC:", roc_auc_lg)

ROC AUC: 0.8647361647361649


In [ ]:
conf_mat_lg = confusion_matrix(y_test, y_pred_lg)
conf_mat_lg

array([[84, 21],
       [19, 55]])

In [ ]:
df.columns

Index(['survived', 'pclass', 'sex', 'age', 'sibsp', 'parch', 'fare',
       'embarked', 'class', 'who', 'adult_male', 'deck', 'embark_town',
       'alive', 'alone'],
      dtype='object')

In [ ]:
x_train= X_train_dop.drop(['who', 'adult_male', 'embark_town', 'alone'], axis = 1)
x_test = X_test_dop.drop(['who', 'adult_male', 'embark_town', 'alone'], axis = 1)

In [ ]:
x_train

,pclass,sex,age,sibsp,parch,fare,embarked,class,deck
331,1,male,45.5,0,0,28.5,S,First,C
733,2,male,23.0,0,0,13.0,S,Second,C
382,3,male,32.0,0,0,7.925,S,Third,C
704,3,male,26.0,1,0,7.8542,S,Third,C
813,3,female,6.0,4,2,31.275,S,Third,C
...,...,...,...,...,...,...,...,...,...
106,3,female,21.0,0,0,7.65,S,Third,C
270,1,male,24.0,0,0,31.0,S,First,C
860,3,male,41.0,2,0,14.1083,S,Third,C
435,1,female,14.0,1,2,120.0,S,First,B


In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
x_train['sex'] = le.fit_transform(x_train['sex'])
x_train['embarked'] = le.fit_transform(x_train['embarked'])
x_train['class'] = le.fit_transform(x_train['class'])
x_train['deck'] = le.fit_transform(x_train['deck'])

In [ ]:
x_test['sex'] = le.fit_transform(x_test['sex'])
x_test['embarked'] = le.fit_transform(x_test['embarked'])
x_test['class'] = le.fit_transform(x_test['class'])
x_test['deck'] = le.fit_transform(x_test['deck'])

In [ ]:
lg = LogisticRegression()
lg.fit(X_train, y_train)
y_pred_lg = lg.predict(X_test)
y_pred_lg_prob = lg.predict_proba(X_test)[:, 1]

In [ ]:
acc_score_lg = accuracy_score(y_test, y_pred_lg)
prec_score_lg = precision_score(y_test, y_pred_lg)
rec_score_lg = recall_score(y_test, y_pred_lg)

In [ ]:
print(f'Accuracy: {acc_score_lg}, Precision: {prec_score_lg}, Recall: {rec_score_lg}')

Accuracy: 0.776536312849162, Precision: 0.7236842105263158, Recall: 0.7432432432432432


In [ ]:
roc_auc_lg = roc_auc_score(y_test, y_pred_lg_prob)
print("ROC AUC:", roc_auc_lg)

ROC AUC: 0.8647361647361649


In [ ]:
conf_mat_lg = confusion_matrix(y_test, y_pred_lg)
conf_mat_lg

array([[84, 21],
       [19, 55]])

Итак, обе модели показли не самые высокие результаты - я склоняюсь к тому, что проблема в самих данных. Для того, чтобы улучшить результаты, возможно, в дальнейшем стоит синтезировать дополнительные признаки; тщательнее отобрать признаки, как я и писала выше, исключив или переработав те, которые дублируют или искажают информацию - это мешает модели устанавливать правильные связи. Также в будущем стоит провести болеее четкий подбор гиперпараметров для сложных моделей и, возможно, использовать пайплайны для подбора моделей.